In [1]:
import pandas as pd

In [57]:
df_menu = pd.read_csv("menu.csv")
df_order = pd.read_csv("order.csv")
df_promotion = pd.read_csv("promotion.csv")

In [36]:
df_order['sales_date'] = df_order['sales_date'].str.replace("/","-")

In [44]:
df_order

,order_id,menu_id,quantity,sales_date
0,1,3,5,2021-01-01
1,1,3,5,2021-01-01
2,1,4,3,2021-01-01
3,1,4,4,2021-01-01
4,1,3,5,2021-01-01
...,...,...,...,...
994,298,3,4,2021-05-01
995,298,3,3,2021-05-01
996,299,4,1,2021-05-01
997,299,4,2,2021-05-01


In [41]:
df_order['sales_date'] = pd.to_datetime(df_order['sales_date'])

In [46]:
df_promotion['start_date'] = pd.to_datetime(df_promotion['start_date'])
df_promotion['end_date'] = pd.to_datetime(df_promotion['end_date'])

In [47]:
df_promotion

,id,start_date,end_date,disc_value,max_disc
0,1,2021-01-01,2021-01-03,0.20,40000
1,2,2021-02-04,2021-02-04,0.30,50000
2,3,2021-03-03,2021-03-06,0.20,30000
3,4,2021-04-01,2021-04-07,0.15,25000
4,5,2021-05-05,2021-05-08,0.25,20000


In [62]:
def convert_discount(sales_date):
    print(sales_date.ilo)
    # test = pd.to_datetime(sales_date)
    # print(test)
    # print(df_promotion[df_promotion['end_date'] >= '2021-01-03'])
    # print(df_promotion[df_promotion['start_date'] == sales_date])
    # print(df_promotion[(df_promotion['start_date'] >= sales_date) & (df_promotion['end_date'] <= sales_date)])

df_order[['sales_date']].apply(convert_discount)

0      2021/01/01
1      2021/01/01
2      2021/01/01
3      2021/01/01
4      2021/01/01
          ...    
994    2021/05/01
995    2021/05/01
996    2021/05/01
997    2021/05/01
998    2021/05/01
Name: sales_date, Length: 999, dtype: object


sales_date    None
dtype: object